# group by를 사용해보자.

- groupby는 전처리 과정에서 많이 사용된다.

In [30]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType
import pyspark.sql.functions as f

In [3]:
sc = SparkContext('local')
sqlCtx = SQLContext(sc)

In [6]:
df = sqlCtx.read.csv( 'c:/Users/FX506HM/Dropbox/pythonedu/data/grade.csv',
                    header=True, inferSchema=True) # infer schema를 쓰지 않으면 전부 strtfh dlstlrgksek. 

df

DataFrame[학년: string, 과목: string, 결과: string, 중간: int, 기말: int]

In [7]:
df.show()

+-----+----+-----+----+----+
| 학년|과목| 결과|중간|기말|
+-----+----+-----+----+----+
|1학년|국어| 좋음|  80|  90|
|1학년|국어| 나쁨|  50|  40|
|1학년|국어| 나쁨|  20|  50|
|1학년|수학| 좋음|  83|  95|
|1학년|수학| 좋음|  93|  86|
|2학년|국어| 나쁨|  44|  65|
|2학년|국어| 좋음|  95|  98|
|2학년|수학| 좋음|  96|  99|
|2학년|수학| 나쁨|  57|  69|
+-----+----+-----+----+----+



# groupby

- groupby를 써서 특정 column 별로 집계해보자.

In [10]:
df.groupBy(df['학년'])

df.groupBy('학년').max().show()

+-----+---------+---------+
| 학년|max(중간)|max(기말)|
+-----+---------+---------+
|2학년|       96|       99|
|1학년|       93|       95|
+-----+---------+---------+



In [11]:
df.groupBy('학년').sum().show()

+-----+---------+---------+
| 학년|sum(중간)|sum(기말)|
+-----+---------+---------+
|2학년|      292|      331|
|1학년|      326|      361|
+-----+---------+---------+



In [12]:
df.groupBy('학년').avg().show()

+-----+---------+---------+
| 학년|avg(중간)|avg(기말)|
+-----+---------+---------+
|2학년|     73.0|    82.75|
|1학년|     65.2|     72.2|
+-----+---------+---------+



In [13]:
df.groupBy('학년').avg('기말').orderBy('학년').show()

+-----+---------+
| 학년|avg(기말)|
+-----+---------+
|1학년|     72.2|
|2학년|    82.75|
+-----+---------+



In [14]:
# 학년 및 과목별 값을 봐보자.
df.groupBy('학년', "과목").avg().orderBy('학년').show()

+-----+----+---------+---------+
| 학년|과목|avg(중간)|avg(기말)|
+-----+----+---------+---------+
|1학년|국어|     50.0|     60.0|
|1학년|수학|     88.0|     90.5|
|2학년|수학|     76.5|     84.0|
|2학년|국어|     69.5|     81.5|
+-----+----+---------+---------+



In [15]:
df.groupBy('학년').agg({'기말': 'sum'}).show()

+-----+---------+
| 학년|sum(기말)|
+-----+---------+
|2학년|      331|
|1학년|      361|
+-----+---------+



In [17]:
df.groupBy('학년').agg(f.sum('중간').alias('중간 합계'), f.avg('중간').alias('중간 평균')).show()

+-----+---------+---------+
| 학년|중간 합계|중간 평균|
+-----+---------+---------+
|2학년|      292|     73.0|
|1학년|      326|     65.2|
+-----+---------+---------+



# fillna

- fillna는 nan data를 채워준다.

In [19]:
df.fillna({'중간':10}).show()

+-----+----+-----+----+----+
| 학년|과목| 결과|중간|기말|
+-----+----+-----+----+----+
|1학년|국어| 좋음|  80|  90|
|1학년|국어| 나쁨|  50|  40|
|1학년|국어| 나쁨|  20|  50|
|1학년|수학| 좋음|  83|  95|
|1학년|수학| 좋음|  93|  86|
|2학년|국어| 나쁨|  44|  65|
|2학년|국어| 좋음|  95|  98|
|2학년|수학| 좋음|  96|  99|
|2학년|수학| 나쁨|  57|  69|
+-----+----+-----+----+----+

